In [9]:
import requests
import os
import pandas as pd

Set the date range to get snapshot data

In [11]:
all_dates = pd.date_range('2019-01-01', '2024-01-04')

Get max blocks

In [12]:
%%time
path = 'D:/blocks/ethereum'
all_days = list()
all_hours = list()
for date in all_dates:
    d = date.strftime("%Y%m%d")
    file = '{}/ethereum_{}.tsv.gz'.format(path, d)
    
    if not os.path.isfile(file):
        print('Downloading block headers on', date.strftime("%Y-%m-%d"))
        url = 'https://gz.blockchair.com/ethereum/blocks/blockchair_ethereum_blocks_{}.tsv.gz'.format(d)
        r = requests.get(url, allow_redirects=True)
        s = open(file, 'wb').write(r.content)
    
    blocks = pd.read_csv(file, compression='gzip', sep='\t', header=0)
    blocks['time'] = pd.to_datetime(blocks['time'])
    
    days = blocks.groupby(pd.Grouper(key='time', freq='1D')).agg({
        'id': ['count', 'max'],
        'fee_total_usd': 'sum',
        'gas_used': 'sum'
    }).reset_index()
    days.columns = ['time', 'count', 'max_block', 'fee_total_usd', 'gas_used']
    days['gas_fee_usd'] = days['fee_total_usd'] / days['gas_used']
    all_days.append(days)
    
    hours = blocks.groupby(pd.Grouper(key='time', freq='1H')).agg({
        'id': ['count', 'max'],
        'fee_total_usd': 'sum',
        'gas_used': 'sum'
    }).reset_index()
    hours.columns = ['time', 'count', 'max_block', 'fee_total_usd', 'gas_used']
    hours['gas_fee_usd'] = hours['fee_total_usd'] / hours['gas_used']
    all_hours.append(hours)

Wall time: 8min 1s


In [4]:
all_days = pd.concat(all_days, ignore_index=True)
all_days.to_csv(f'data/blocks_daily.csv', index=False)

In [5]:
all_hours = pd.concat(all_hours, ignore_index=True)
all_hours.to_csv(f'data/blocks_hourly.csv', index=False)